In [1]:
import pandas as pd
import numpy as np
import random
import scipy.stats as sps
import matplotlib.pyplot as plt
%matplotlib inline

Известно, что сопряженным к экспоненциальному является гамма распределение $Г(\alpha, \beta)$ со следующими гиперпараметрами апостериорного распределения: $\alpha + n, \beta + \sum_{i=1}^n X_i$.

Загружаем данные из файла:

In [2]:
times_of_breaking = pd.read_csv('/home/denis/Downloads/6.csv')
max_time = 70000
print(len(times_of_breaking))

1000


In [3]:
times_of_breaking = np.asarray(times_of_breaking['Values'])

Выберем параметры априорного распределения таковыми, чтобы у плотности Гамма распределения не было горба(т.к. мы не знаем ничего, как распределена $\lambda$), и возьмем $\beta$ так, чтобы график плотности был гладким. Если брать параметр $\alpha$ > 1, то в окрестности некоторой точки образуется горб. Следовательно нам подходит модель с априорным распределением $\Gamma(1,2)$. Оценкой $\theta $ будет $\frac{\alpha + n}{\beta + \sum_{i=1}^n {X_{i}}}$. Это в случае того, когда в качестве априорного распределения мы взяли $\Gamma(\alpha, \beta)$. Зададим функцию, с помощью которой мы будем оценивать параметр $\theta$:

In [4]:
def evaluation(alpha, beta, cumulative_sum):
    return [(float(alpha) + float(i)) / 
            (float(beta) + float(cumulative_sum[i])) for i in range(len(times_of_breaking))]

Запишем нашу выборку из экспоненциального распределения(разности между поломками):

In [5]:
sample = [times_of_breaking[0]]
for i in range(len(times_of_breaking) - 1):
    sample.append(times_of_breaking[i + 1] - times_of_breaking[i])

Теперь с помощью этой выборки мы можем задать массив значений $\lambda$:

In [6]:
lambda_values = evaluation(1, 2, np.cumsum(sample))

Из задачи из раздела УМОII используем функцию, считающую условное математическое ожидание:

In [7]:
def conditional_expectation(t, s, lambda_value) :
    if s > max_time:
        return float(t - s) / float(lambda_value) + len(times_of_breaking)
    return float(t - s) / float(lambda_value) + np.where(times_of_breaking >= s)[0][0]

Ну и выведем наши значения: break, если была поломка; УМО, если поломки не было. УМО считаем с разными lambda, которые мы оценили выше:

In [8]:
events = [(t, True) for t in times_of_breaking] + [(i, False) for i in range(1,69000,200)] 
events.sort()
short_events = []
for i in range(1, len(events), 10):
    short_events.append(events[i])

In [9]:
t = 70000
i = -1
print ('Time' + '\tE')
for event in short_events:
    if event[1]:
        print (str(event[0]) + '\tbreak')
        i += 1
    else:
        if i != -1:
            print (str(event[0]) + '\t' + str(conditional_expectation(t, event[0], lambda_values[i])))
        else:
            print (str(event[0]) + '\t' + '0')

Time	E
158.377	break
601	11130011.423
1021.8233	break
1318.8093	break
1655.0349	break
2201	5596684.7324
2625.1881	break
3004.0609	break
3472.1577	break
3804.1451	break
4275.2071	break
5095.8533	break
5601	4596152.38751
6076.7039	break
6690.8602	break
7308.3547	break
7893.0347	break
8409.5739	break
9001	4125223.8745
9374.8468	break
10017.7219	break
10554.895	break
11001	3827753.98564
11662.2666	break
12201	3646221.53439
12813.9724	break
13241.5337	break
13674.1624	break
14263.5472	break
14601	3058439.82775
15161.1543	break
15601	2951119.67476
16201	2918579.53226
16601	2896889.43726
17001	2875199.34226
17483.3187	break
17828.2307	break
18411.0236	break
18801	2603329.30204
19401	2572831.30426
19963.4404	break
20420.9631	break
20899.0075	break
21201	2554573.64443
21640.2849	break
22279.9196	break
22661.1542	break
23401	2340237.91834
23920.9894	break
24159.1174	break
24622.3048	break
25166.1364	break
25661.8035	break
26348.0704	break
26746.3207	break
27103.0635	break
27582.1738	break
28121.